<a href="https://colab.research.google.com/github/PierceRoberts34/MLOPS-Proof-of-Concept/blob/main/TimeGAN_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install kaleido==0.2.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 10.7 MB/s eta 0:00:00


In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [4]:
# Run on GPU for best results
tf.config.list_physical_devices('GPU')

[]

In [6]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [7]:
# We import the right wrist accelerometer data.
def importData(filename):
  df=pd.read_csv(f'/content/drive/MyDrive/Sensor Data/PAMAP2/Protocol/{filename}.dat',
                     sep=' ',
                     header=None,
                     usecols=[1,4,5,6])
  df = df[df[1].isin([1, 2, 3, 4, 12, 13, 16, 17])].copy() # Use only the activities which are common to all subjects
  df = df.drop(columns=[1]) # We no longer need the activity column
  df = df.dropna() # Ensure the data has no missing values
  df = df.iloc[::3, :] # Resample from 100Hz to 33Hz for performance
  return df

We keep only the rows common to each subject. Our final dataset consists of timeseries data in three axes.

In [8]:
subject_files = ['subject101', 'subject102', 'subject103', 'subject104', 'subject105', 'subject106', 'subject107']

x_train = pd.concat((importData(f) for f in subject_files), ignore_index=True).to_numpy()
x_test = importData('subject108').to_numpy()

In [9]:
!git clone https://github.com/PierceRoberts34/time-gan-tensorflow.git
%cd time-gan-tensorflow

Cloning into 'time-gan-tensorflow'...
remote: Enumerating objects: 133, done.
remote: Counting objects: 100% (133/133), done.
remote: Compressing objects: 100% (118/118), done.
remote: Total 133 (delta 77), reused 37 (delta 14), pack-reused 0 (from 0)
Receiving objects: 100% (133/133), 7.96 MiB | 12.33 MiB/s, done.
Resolving deltas: 100% (77/77), done.
/content/time-gan-tensorflow


In [10]:
from time_gan_tensorflow.model import TimeGAN
from time_gan_tensorflow.plots import plot

# Fit the model to the training data
model = TimeGAN(
    x=x_train,
    timesteps=33,
    hidden_dim=64,
    num_layers=3,
    lambda_param=0.1,
    eta_param=10,
    learning_rate=0.001,
    batch_size=16
)

model.fit(
    epochs=500,
    verbose=True
)

# Reconstruct the test data
x_hat = model.reconstruct(x=x_test)

# Generate the synthetic data
x_sim = model.simulate(samples=len(x_test))

# Plot the actual, reconstructed and synthetic data
fig = plot(actual=x_test, reconstructed=x_hat, synthetic=x_sim)
fig.write_image('results.png', scale=4, height=900, width=700)

epoch: 1 autoencoder_loss: 0.002759 generator_loss: 32.321564 discriminator_loss: 30.221428


In [11]:
print("Real Data")
print(x_train)
print("Synthetic Data")
print(x_sim)
print(x_sim.shape[0])

Real Data
[[ 2.2153    8.27915   5.58753 ]
 [ 2.218     7.14365   5.8993  ]
 [ 2.41148   7.5978    5.93915 ]
 ...
 [-6.28978   7.45238  -0.964473]
 [-6.37124   7.37574  -1.11938 ]
 [-6.48648   7.3004   -1.15934 ]]
Synthetic Data
[[-3.80792581  3.6744547   5.28285941]
 [-3.81155338  3.11784052  5.63324672]
 [-4.21640256  2.12019925  6.08012234]
 ...
 [-6.18169371  7.3973421   2.80722285]
 [-6.23242586  7.39146864  2.77235387]
 [-6.27638609  7.38961491  2.74149681]]
60588


In [12]:
synthetic_df = pd.DataFrame(x_sim)
index = 0

In [19]:
# Generate new data
append = synthetic_df[index:index+5]
index += 5
print(append)
append.to_csv('/content/drive/MyDrive/Sensor Data/Test Data/synthetic_data.csv', mode='a', header=None, index=False)

            0         1         2
30 -10.152992 -2.498199  4.210392
31 -10.169897 -2.501760  4.194969
32 -10.188829 -2.500569  4.180388
33  -3.748180  3.642691  5.323518
34  -3.612217  3.025770  5.781303
